<a href="https://colab.research.google.com/github/HananeNourMoussa/darija-ner/blob/master/Experiment_3_BERT_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Experiment Parameters: 

In [ ]:
# UBC-NLP/ARBERT
# UBC-NLP/MARBERT
# bert-base-multilingual-cased
# SI2M-Lab/DarijaBERT
# CAMeL-Lab/bert-base-arabic-camelbert-da
# CAMeL-Lab/bert-base-arabic-camelbert-msa
# CAMeL-Lab/bert-base-arabic-camelbert-mix
# aubmindlab/bert-base-arabertv02

In [ ]:
config_default = {
    'epochs' : 5, 
    'batch_size': 8, 
    'lr_bert': 3e-5,
    'lr_rest':1e-4, 
    'hidden_size': 256,
    'num_layers': 1,
    'eps': 1e-8,
    "lm": "aubmindlab/bert-base-arabertv02", 
    "p1":0.3, 
    "p2": 0.2,
    "dataset": "mixedNERcorp"
}

In [ ]:
MAX_LEN = 100
lm = "aubmindlab/bert-base-arabertv02"
batch_size = 8
freeze_bert = False

In [ ]:
#GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 29 21:25:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Installations and Imports: 

In [ ]:
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q
!pip install torch -q
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import transformers
from transformers import BertForTokenClassification, AutoTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from google.colab import drive
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
from tqdm import tqdm, trange
from statistics import mean
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import LongTensor
import wandb
import os 
import random

In [ ]:
#ensure deterministic behavior 
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Data Preprocessing: 

In [ ]:
drive.mount('/content/gdrive/', force_remount=True)
dataset = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/MixedNERcorp_train.csv")
dataset.pop("Unnamed: 0")
dataset

Mounted at /content/gdrive/


,Sentence,Token,Tag
0,0,Uppsala,B-LOC
1,0,),O
2,0,هيّا,O
3,0,رابع,O
4,0,أكبر,O
...,...,...,...
177821,5957,في,O
177822,5957,مختلف,O
177823,5957,أنحاء,O
177824,5957,المصنع,O


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(lm)

In [ ]:
class SentenceGetter(object): 
  def __init__(self, data): 
    #start from the first sentence
    self.n_sent = 1
    self.data = data
    self.empty = False
    #aggregate token and tag
    agg_func = lambda s:[(to, ta) for to, ta in zip (s["Token"].values.tolist(),
                                                     s["Tag"].values.tolist())]
    self.grouped = self.data.groupby("Sentence").apply(agg_func)
    #make list of sentences
    self.sentences = [s for s in self.grouped]  
  def get_next(self): 
    try: 
      #get current sentence
      s = self.grouped["Sentence: {}".format(self.n_sent)]
      #move to next sentence
      self.n_sent += 1
      return s
    except: 
      return None

In [ ]:
getter = SentenceGetter(dataset)

In [ ]:
#get tokens of sentences
sentences = [[token[0] for token in sentence] for sentence in getter.sentences]
sentences[0]

['Uppsala',
 ')',
 'هيّا',
 'رابع',
 'أكبر',
 'مدينة',
 'ف',
 'سّويد',
 'من',
 'بعد',
 'سطوكهولم',
 '،',
 'ݣوتنبورݣ',
 'ؤ',
 'مالمو',
 '.']

In [ ]:
#remove a nan value that causes errors later
sentences_ = list()
for sentence in sentences: 
  sentence_ = list(filter(lambda item: type(item) == str, sentence))
  sentences_.append(sentence_)

In [ ]:
#getting tags 
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-LOC', 'O']


In [ ]:
#adding PAD to tags and assigning label numbers 
tag_values = list(set(dataset["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
#function to tokenize each word into subwords while preserving corresponding labels
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
#pass each sentence at a time to the function
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences_, labels)
]

In [ ]:
#separate text and labeLs
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
#convert text to input ids
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
#getting encoded tags with PAD included for CLS and SEP tokens
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
#setting attention masks to tell model to ignore PADs 
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
#split data (sentences) to train, validation, and test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.2)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.2)

In [ ]:
#convert to pyTorch tensors
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# Model Definitions: 

### BERT-CRF: 

In [ ]:
!pip install pytorch-crf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchcrf import CRF

In [ ]:
class BERT_CRF(nn.Module): 
  def __init__(self, p1, p2, num_classes = len(tag2idx)):
    super().__init__()
    config = transformers.BertConfig.from_pretrained(lm, hidden_dropout_prob=p1)
    self.bert = transformers.BertModel.from_pretrained(lm, config =config)
    input_size = self.bert.config.to_dict()['hidden_size']
    self.fc = nn.Linear(input_size, num_classes)
    self.crf = CRF(num_classes, batch_first = True)
    self.drop = nn.Dropout(p2)
    if freeze_bert == True: 
      for param in self.bert.parameters(): 
        param.requires_grad = False
    else: 
      for param in self.bert.parameters(): 
        param.requires_grad = True
  def forward (self, input_ids, tags, token_type_ids = None, input_mask = None):
    emissions = self.tag_outputs(input_ids, token_type_ids, input_mask)
    loss = -1*self.crf(emissions, tags, mask=input_mask.byte())
    return loss
  def tag_outputs(self, input_ids, token_type_ids = None, input_mask = None): 
    s = self.bert(input_ids = input_ids, attention_mask=input_mask)
    s = s['last_hidden_state']
    #s = self.drop(s)
    emissions = self.fc(s)
    return emissions
  def predict(self, input_ids, token_type_ids = None, input_mask = None): 
    emissions = self.tag_outputs(input_ids, token_type_ids, input_mask)
    if (input_mask == None): 
      return self.crf.decode(emissions)
    return self.crf.decode(emissions, input_mask.byte())

# Training and Validation: 

In [ ]:
model = BERT_CRF(p1 = 0.3, p2 = 0.2)
optimizer = optim.AdamW([
        {'params': model.bert.parameters(), 'lr':3e-5},
        {'params': model.fc.parameters(), 'lr':1e-4}
    ])
total_steps = len(train_dataloader) * 5
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
model.cuda()

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
def get_metrics(true, pred): 
  acc = accuracy_score(true, pred)
  f1 = f1_score(true, pred)
  prec = precision_score(true, pred)
  recall = recall_score(true, pred)
  report = classification_report(true, pred)
  return acc, f1, prec, recall, report

#### CRF-Code:

In [ ]:
def evaluate_model(model, dataset, total_loss = 0): 
  model.eval()
  predictions = []
  truth = []
  with torch.no_grad():
      for batch in dataset:
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, labels = batch
        logits = model.predict(input_ids = input_ids, input_mask = input_mask)
        label_ids = labels.to('cpu').numpy()
        predictions.extend(logits)
        truth.extend(label_ids)
  pred_tags = [tag_values[p_i] for p, l in zip(predictions, truth)
                                for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
  true_tags = [tag_values[l_i] for l in truth
                                for l_i in l if tag_values[l_i] != "PAD"]
  pred = list()
  pred.append(pred_tags)   
  true = list()
  true.append(true_tags)
  return pred, true

In [ ]:
training_loss = []
validation_loss = []
tr_accuracies, val_accuracies = [], []
tr_f1s, val_f1s = [], []
tr_precisions, val_precisions = [], []
tr_recalls, val_recalls = [], []
def train(config = None): 
  with wandb.init(project="final-results", config=config_default):
    config = wandb.config

    
    # Train the model
    for epoch in trange(config.epochs, desc = "Epoch"):
        # Training phase
        model.train()
        train_loss = 0
        for step, batch in enumerate(train_dataloader):
            model.zero_grad()
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, labels = batch
            loss = model(input_ids = input_ids, input_mask = input_mask, tags = labels)
            loss.backward()
            train_loss += loss.item()
            #torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
            optimizer.step()
            scheduler.step()
        avg_train_loss = train_loss / len(train_dataloader)
        print("Average train loss: {}".format(avg_train_loss))
        training_loss.append(avg_train_loss)
        #log train loss
        wandb.log({'train_loss': avg_train_loss, "epoch": epoch})
        #performance on training
        tr_pred, tr_true = evaluate_model(model, train_dataloader)
        tr_acc, tr_f1, tr_prec, tr_recall, tr_report = get_metrics(tr_true, tr_pred)  
        print("Training Accuracy: {}".format(tr_acc))
        tr_accuracies.append(tr_acc)
        wandb.log({'train_acc': tr_acc, "epoch": epoch})
        print("Training F1-Score: {}".format(tr_f1))
        tr_f1s.append(tr_f1)
        wandb.log({'train_f1': tr_f1, "epoch": epoch})
        print("Training Precision: {}".format(tr_prec))
        tr_precisions.append(tr_prec)
        wandb.log({'train_prec': tr_prec, "epoch": epoch})
        print("Training Recall: {}".format(tr_recall))
        tr_recalls.append(tr_recall)
        wandb.log({'train_recall': tr_recall, "epoch": epoch})
        print("Training Classification Report:\n {}".format(tr_report))
        print()
        # performance on validation
        val_pred, val_true = evaluate_model(model, valid_dataloader)
        val_acc, val_f1, val_prec, val_recall, val_report = get_metrics(val_true, val_pred)   
        print("Validation Accuracy: {}".format(val_acc))
        val_accuracies.append(val_acc)
        wandb.log({'val_acc': val_acc, "epoch": epoch})
        print("Validation F1-Score: {}".format(val_f1))
        val_f1s.append(val_f1)
        wandb.log({'val_f1': val_f1, "epoch": epoch})
        print("Validation Precision: {}".format(val_prec))
        val_precisions.append(val_prec)
        wandb.log({'val_prec': val_prec, "epoch": epoch})
        print("Validation Recall: {}".format(val_recall))
        val_recalls.append(val_recall)
        wandb.log({'val_recall': val_recall, "epoch": epoch})
        overfitting = abs(tr_f1-val_f1)
        wandb.log({'overfitting': overfitting, "epoch": epoch})
        print("Validation Classification Report:\n {}".format(val_report))
        print()
  #model.save(os.path.join(wandb.run.dir, "model.h5"))
#wandb.agent(sweep_id, train)
train()

wandb: Currently logged in as: hanane-n-moussa (darijaner). Use `wandb login --relogin` to force relogin


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Average train loss: 92.74137924421554
Training Accuracy: 0.9538074114014019
Training F1-Score: 0.7419462638684668
Training Precision: 0.754041570438799
Training Recall: 0.7302328640968294
Training Classification Report:
               precision    recall  f1-score   support

         LOC       0.80      0.85      0.83      6509
        MISC       0.62      0.38      0.47      3239
         ORG       0.59      0.70      0.64      2014
         PER       0.84      0.85      0.85      3440

   micro avg       0.75      0.73      0.74     15202
   macro avg       0.72      0.70      0.70     15202
weighted avg       0.75      0.73      0.73     15202




Epoch:  20%|██        | 1/5 [03:24<13:36, 204.01s/it]

Validation Accuracy: 0.951170503788704
Validation F1-Score: 0.7164453792559484
Validation Precision: 0.7258923814597762
Validation Recall: 0.7072411108227356
Validation Classification Report:
               precision    recall  f1-score   support

         LOC       0.77      0.83      0.80      1509
        MISC       0.61      0.34      0.44       833
         ORG       0.56      0.70      0.63       564
         PER       0.82      0.84      0.83       947

   micro avg       0.73      0.71      0.72      3853
   macro avg       0.69      0.68      0.67      3853
weighted avg       0.72      0.71      0.70      3853


Average train loss: 48.6462621896859
Training Accuracy: 0.9648424096857997
Training F1-Score: 0.7967537506466632
Training Precision: 0.783479587943533
Training Recall: 0.8104854624391528
Training Classification Report:
               precision    recall  f1-score   support

         LOC       0.84      0.87      0.85      6509
        MISC       0.65      0.66      0.6

...

Epoch:  80%|████████  | 4/5 [13:25<03:20, 200.16s/it]

Validation Accuracy: 0.9644370203597453
Validation F1-Score: 0.8038105046343975
Validation Precision: 0.7974457215836526
Validation Recall: 0.8102777056838827
Validation Classification Report:
               precision    recall  f1-score   support

         LOC       0.86      0.87      0.87      1509
        MISC       0.66      0.64      0.65       833
         ORG       0.70      0.78      0.74       564
         PER       0.88      0.88      0.88       947

   micro avg       0.80      0.81      0.80      3853
   macro avg       0.77      0.79      0.78      3853
weighted avg       0.80      0.81      0.80      3853


Average train loss: 30.280353378929547
Training Accuracy: 0.9767045733052301
Training F1-Score: 0.8630965863128944
Training Precision: 0.8493281538559511
Training Recall: 0.8773187738455467
Training Classification Report:
               precision    recall  f1-score   support

         LOC       0.88      0.92      0.90      6509
        MISC       0.76      0.79     

Epoch: 100%|██████████| 5/5 [16:41<00:00, 200.30s/it]

Validation Accuracy: 0.9651390805432494
Validation F1-Score: 0.8078703703703705
Validation Precision: 0.8006627580932959
Validation Recall: 0.8152089281079679
Validation Classification Report:
               precision    recall  f1-score   support

         LOC       0.86      0.88      0.87      1509
        MISC       0.68      0.64      0.66       833
         ORG       0.70      0.78      0.74       564
         PER       0.88      0.89      0.88       947

   micro avg       0.80      0.82      0.81      3853
   macro avg       0.78      0.80      0.79      3853
weighted avg       0.80      0.82      0.81      3853




epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
overfitting,▁▃▇██
train_acc,▁▄▇██
train_f1,▁▄▇██
train_loss,█▃▂▁▁
train_prec,▁▃▇██
train_recall,▁▅▇██
val_acc,▁▄▇██
val_f1,▁▄▇██
val_prec,▁▃▇██
val_recall,▁▅▇██


# Evaluation on Test Set: 

### On DarNERcorp_test: 

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/MyDrive/Capstone/AraBERT-CRF-msa-darija-ner')

In [ ]:
darner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/DarNERcorp_test.csv")
test_darner_set = ' '.join(darner_test["Token"])
tokenized_sentence = tokenizer.encode(test_darner_set)

Token indices sequence length is longer than the specified maximum sequence length for this model (17957 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
new_labels = []
new_tokens = []
n = len(tokenized_sentence)//512
for i in range(0, n+1):
  if (i == n): 
    batch = tokenized_sentence[i*512:len(tokenized_sentence)]
  else: 
    batch = tokenized_sentence[i*512:i*512+512]
  input_ids = torch.tensor([batch]).cuda()
  with torch.no_grad():
      label_indices = model.predict(input_ids)
  #label_indices = np.argmax(logits.to('cpu').numpy(), axis=1)
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          #adding subword to previous word
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)

In [ ]:
dar_predictions = [new_labels[1:len(new_labels)-1]]
dar_tags = [list(darner_test['Tag'])]

In [ ]:
dar_acc, dar_f1, dar_prec, dar_recall, dar_report = get_metrics(dar_tags, dar_predictions)
print(dar_acc,'\n', dar_prec,'\n', dar_recall,'\n', dar_f1,'\n', dar_report)

0.9427162367223065 
 0.6561181434599156 
 0.7300469483568075 
 0.691111111111111 
               precision    recall  f1-score   support

         LOC       0.73      0.87      0.79       539
        MISC       0.56      0.57      0.56       395
         ORG       0.56      0.64      0.60       190
         PER       0.71      0.78      0.75       154

   micro avg       0.66      0.73      0.69      1278
   macro avg       0.64      0.71      0.68      1278
weighted avg       0.65      0.73      0.69      1278



### On mixedNERcorp_test:

In [ ]:
import unicodedata as ud
def remove_punc(s): 
  if (len(s) > 1) and s!= '--': 
    s = ''.join(c for c in s if not ud.category(c).startswith('P'))
  return s

In [ ]:
mixner_test = pd.read_csv("/content/gdrive/MyDrive/annotated_corpus/ner-corpus-darija/MixedNERcorp_test.csv")

In [ ]:
tokens = list(mixner_test["Token"])
tokens = list(map(remove_punc, tokens))

In [ ]:
test_mixner_set = ' '.join(tokens)
tokenized_sentence = tokenizer.encode(test_mixner_set)

In [ ]:
new_labels = []
new_tokens = []
n = len(tokenized_sentence)//512
for i in range(0, n+1):
  if (i == n): 
    batch = tokenized_sentence[i*512:len(tokenized_sentence)]
  else: 
    batch = tokenized_sentence[i*512:i*512+512]
  input_ids = torch.tensor([batch]).cuda()
  with torch.no_grad():
      label_indices = model.predict(input_ids)
  #label_indices = np.argmax(logits.to('cpu').numpy(), axis=1)
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          #adding subword to previous word
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)

In [ ]:
mix_predictions = [new_labels[1:len(new_labels)-1]]
mix_tags = [list(mixner_test['Tag'])]

In [ ]:
mix_acc, mix_f1, mix_prec, mix_recall, mix_report = get_metrics(mix_tags, mix_predictions)
print(mix_acc,'\n', mix_prec,'\n', mix_recall,'\n', mix_f1,'\n', mix_report)

0.9568450822247827 
 0.7426067907995619 
 0.7611563289362896 
 0.7517671517671517 
               precision    recall  f1-score   support

         LOC       0.78      0.86      0.82      1215
        MISC       0.58      0.56      0.57       638
         ORG       0.68      0.69      0.69       649
         PER       0.84      0.81      0.82      1061

   micro avg       0.74      0.76      0.75      3563
   macro avg       0.72      0.73      0.72      3563
weighted avg       0.74      0.76      0.75      3563



In [ ]:
model

BERT_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("AraBERT-CRF-msa-darija-ner")

AttributeError: ignored

In [ ]:
from transformers import AutoModel
model.save_pretrained("darija-ner", push_to_hub=True, repo_name="AraBERT-CRF-msa-darija-ner")

AttributeError: ignored